<a href="https://colab.research.google.com/github/vannorman/zannorman/blob/master/example_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
prompt = "a tree"  # YOUR TEXT TO GENERATE HERE

In [ ]:
!git clone https://github.com/openai/shap-e
!mv shap-e/* .
!pip install -e .

Cloning into 'shap-e'...
remote: Enumerating objects: 322, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 322 (delta 23), reused 8 (delta 8), pack-reused 286
Receiving objects: 100% (322/322), 11.71 MiB | 21.22 MiB/s, done.
Resolving deltas: 100% (45/45), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-xf6defvt/clip_0e7c67c189144e9080e320f8c8471a87
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-xf6defvt/clip_0e7c67c189144e9080e320f8c8471a87
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.1 MB/s eta 0:00:00
  Preparing metadata

In [ ]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from shap_e.util.image_util import load_image

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|████████████████████████████████████████| 890M/890M [00:08<00:00, 104MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [ ]:
batch_size = 4
guidance_scale = 15.0

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
render_mode = 'nerf' # you can change this to 'stf'
size = 256+128 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhgAGAAYQAAAAAADMAAAAzADMzAGYzADNmAGZmADOZAAAAMzMAMwAzMzMzM2…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgAGAAYQAAAAAADMAAGYAAAAzADMzAGYzAABmADNmAGZmAAAAMzMAM2YAMw…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgAGAAYMAAAAAADMAAAAzADMzAGYzAABmADNmAAAAMzMAMwAzMzMzMwBmMz…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgAGAAYQAAAAAADMAAGYAAAAzADMzAGYzAABmADNmAAAAMzMAM2YAMwAzMz…

In [ ]:
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        decode_latent_mesh(xm, latent).tri_mesh().write_ply(f)


/content/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/content/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(
